Initialisation

In [1]:
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt
import time

N = 10 #Number of points

Dataset generation

In [2]:
def pointgeneration():
    """
    Return a list of two numbers in [0,1]
    """
    return rd.rand(2)

In [11]:
def generateB(n):
    """
    Return list of n list of 2 elements
    """
    return rd.uniform(size=(n, 2))

In [4]:
def generateA(n):

    A=generateB(n-4)

    #adding the corner
    A= np.append(A,[[0,0]],axis=0)
    A= np.append(A,[[0,1]],axis=0)
    A= np.append(A,[[1,0]],axis=0)
    A= np.append(A,[[1,1]],axis=0)

    #rotating the set
    angle = rd.uniform() * 2 * np.pi
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    A = A @ rotation_matrix

    # Shuffle the points
    np.random.shuffle(A)
    return A


In [5]:
def generateC(n,r):
    C=[]
    i=0
    while i<n:
        x,y=rd.uniform(-r,r),rd.uniform(-r,r)
        if x**2+y**2<=r**2:
            C.append(np.array([x,y]))
            i+=1
    return np.array(C)

In [6]:
def generateD(n,r):
    D=[]
    for i in range(n):
        angle = rd.uniform()*2*np.pi
        D.append( np.array([r*np.cos(angle),r*np.sin(angle)]))
    return np.array(D)


Sweeping

In [7]:
def clockwise(a,b,c):
    #Takes 3 points a,b,c of type np.array and return if the triangle ABC is clock-wised 
    
    if np.linalg.det(np.concatenate(((c-b).reshape(2,1),(b-a).reshape(2,1)),axis=1))>0: return True
    else: return False

In [8]:
def sweeping_algo(l):
    #l is a list of points of type np.array, assuming there are >= 2 points and sorted by increasing x
    order=l[:, 0].argsort()
    l=l[order]
    N=l.shape[0]

    hull=[]

    hull.append(l[0])
    hull.append(l[1])

    for i in (range(2,N)):
        if clockwise(hull[-2],hull[-1],l[i]): hull.append(l[i])
        else : 
            while len(hull)>=2 and not clockwise(hull[-2],hull[-1],l[i]): 
                hull.pop()
            hull.append(l[i])

    hull2=[]
    hull2.append(l[N-1])
    hull2.append(l[N-2])

    for i in reversed(range(N-2)):
        if clockwise(hull2[-2],hull2[-1],l[i]): hull2.append(l[i])
        else : 
            while len(hull2)>=2 and not clockwise(hull2[-2],hull2[-1],l[i]): 
                hull2.pop()
            hull2.append(l[i])
    
    return np.array(hull+hull2)


Test Sweeping

In [9]:
def sweeping_test(N):
    c=generateC(N,1)

    cHull=sweeping_algo(c)

    plt.plot(c[:,0],c[:,1],'xb')
    plt.plot(cHull[:,0],cHull[:,1],'-r')

    plt.show()


In [28]:
def complexity_sweeping(N):
    R=range(int(1e6),N,int(1e6))
    T=[]
    for i in R:
        start_time = time.time()
        sweeping_algo(generateC(i,1))
        T.append( time.time() - start_time )
    plt.plot(R,T,'-r')
    plt.show()

In [29]:
complexity_sweeping(int(1e8))

KeyboardInterrupt: 